# 分割
SQLでは便利なパッケージがないので、分割はRかPythonでやる。

In [1]:
import sys
sys.path.append("../src")
from data_loader import load_production, load_monthly_index

production_tb = load_production()
monthly_index_tb = load_monthly_index()

## レコードデータにおけるモデル検証用のデータ分割
製造レコードのデータを用いて、予測モデル構築のためのデータ分割を行う。

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [3]:
# ホールドアウト検証用のデータ分割
# 予測モデルの入力値と予測対象の値を別々にtrain_test_split関数に設定
# test_sizeは検証データの割合
train_data, test_data, train_target, test_target = \
    train_test_split(production_tb.drop('fault_flg', axis=1),
                     production_tb[['fault_flg']],
                     test_size=0.2)

# train_test_splitによって、行名を現在の行番号に直す
train_data.reset_index(inplace=True, drop=True)
test_data.reset_index(inplace=True, drop=True)
train_target.reset_index(inplace=True, drop=True)
test_target.reset_index(inplace=True, drop=True)

# 対象の行番号リストを生成
row_no_list = list(range(len(train_target)))

# 交差検証用のデータ分割
k_fold = KFold(n_splits=4, shuffle=True)

# 交差数分繰り返し処理、並列処理も可能な部分
for train_cv_no, test_cv_no in k_fold.split(row_no_list):

    # 交差検証における学習データを抽出
    train_cv = train_data.iloc[train_cv_no, :]

    # 交差検証における検証データを抽出
    test_cv = train_data.iloc[test_cv_no, :]


In [4]:
train_cv

,type,length,thickness
0,C,321.437280,22.168721
1,C,278.695027,38.982026
2,D,126.517115,3.932203
5,B,256.560480,24.036465
6,C,439.305770,38.753562
...,...,...,...
794,C,224.952437,44.516058
795,A,178.750063,35.397299
796,D,196.474358,3.077979
797,C,439.075539,48.683697


## 時系列データにおけるモデル検証用のデータ分割
時系列データにおいて単純な交差検証は有効でない。</br>
方法の一つとして、学習データと検証データを時間軸に対してスライドしながら検証する方法がある。</br>
データ期間が足りない場合は、学習データをスライドせずに増やしていく検証もある。

月ごとの経営指標を対象とする。月ごとのレコードデータを対象に、学習データと検証データを時間軸にして1ヶ月ごとスライドしながら生成する。学習期間は24ヶ月、検証期間は12ヶ月、スライドする期間は12ヶ月とする。

In [5]:
# train_window_startに、最初の学習データの開始行番号を指定
train_window_start = 0
# train_window_endに、最初の学習データの終了行番号を指定
train_window_end = 24
# horizonに、検証データのデータ数を指定
horizon = 12
# skipにスライドするデータ数を設定
skip = 12

# 年月に基づいてデータを並び替え
monthly_index_tb.sort_values(by='year_month')

while True:
    # 検証データの終了行番号を計算
    test_window_end = train_window_end + horizon

    # 行番号を指定して、元データから学習データを取得
    # train_window_startの部分を1に固定すれば、学習データを増やしていく検証に変更可能
    train = monthly_index_tb[train_window_start:train_window_end]

    # 行番号を指定して、元データから検証データを取得
    test = monthly_index_tb[train_window_end:test_window_end]

    # 検証データの終了行番号が元データの行数以上になっているか判定
    if test_window_end >= len(monthly_index_tb.index):
        # 全データを対象にした場合終了
        break

    # データをスライドさせる
    train_window_start += skip
    train_window_end += skip

# 交差検定の結果をまとめる
